In [2]:
import os
import cv2
import numpy as np
import requests
import math
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageRegionCreateEntry
from msrest.authentication import ApiKeyCredentials
from dotenv import load_dotenv
import keras_ocr

pipeline = keras_ocr.pipeline.Pipeline()

load_dotenv()

training_key = os.getenv('training_key')
endpoint = os.getenv('endpoint')
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
project_id = os.getenv('project_id')
trainer = CustomVisionTrainingClient(endpoint, credentials)

def draw_boxes(image, prediction_groups):
    for box in prediction_groups[0]:
        points = np.array([box[1][0], box[1][1], box[1][2], box[1][3]], dtype=np.int32)
        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
    return image

def inpaint_text(image, pipeline):
    temp_image = image.copy()  # Create a copy to avoid modifying the original image
    prediction_groups = pipeline.recognize([temp_image])
    mask = np.zeros(temp_image.shape[:2], dtype="uint8")
    image_with_boxes = draw_boxes(temp_image, prediction_groups)
    for box in prediction_groups[0]:
        print(f"box", box)

        points = np.array([box[1][0], box[1][1], box[1][3]], dtype=np.int32)
        cv2.fillConvexPoly(mask, points, 255)

        #Old
        # x0, y0 = box[1][0]
        # x1, y1 = box[1][1] 
        # x2, y2 = box[1][2]
        # x3, y3 = box[1][3] 
        
        # x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        # x_mid1, y_mid1 = midpoint(x0, y0, x3, y3)
        
        # thickness = int(math.sqrt((x2 - x1)**2 + (y2 - y1)**2))
        
        # cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mid1), 255, thickness)
    
    # Apply inpainting
    masked_image = cv2.inpaint(temp_image, mask, 7, cv2.INPAINT_NS)
    return masked_image, image_with_boxes

def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2) / 2)
    y_mid = int((y1 + y2) / 2)
    return x_mid, y_mid

def return_as_dict(tag):
    return tag.as_dict()

remote_tags = trainer.get_tags(project_id)
tags = list(map(return_as_dict, remote_tags))
tag_RA660 = {}

for tag in tags:
    if tag['name'] == 'Tremetrics_RA660':
        tag_RA660 = tag

images = trainer.get_images(project_id, tag_ids=[tag_RA660['id']], order_by=None, tagging_status="Tagged")
image_dir = "C:\\Users\\nleo\\Pictures\\Training data\\Tremetrics\\Tremetrics_RA660"
original_dir = f'{image_dir}\\original'
masked_dir = f'{image_dir}\\masked'
os.makedirs(original_dir, exist_ok=True)
os.makedirs(masked_dir, exist_ok=True)

for image_data in images:
    image_id = image_data.id
    image_url = image_data.original_image_uri

    resp = requests.get(image_url, stream=True).raw
    image = np.asarray(bytearray(resp.read()), dtype='uint8')
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    original_image_path = os.path.join(original_dir, f'original_{image_id}.png')
    cv2.imwrite(original_image_path, image)
    
    keras_read_image = keras_ocr.tools.read(original_image_path)
    masked_image, image_with_boxes = inpaint_text(keras_read_image, pipeline)

    # Save masked image
    masked_image_path = os.path.join(masked_dir, f'processed_{image_id}.png')
    # cv2.imwrite(masked_image_path, masked_image)

    window_name = 'image with boxes'
    cv2.imshow(window_name, image_with_boxes)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Looking for C:\Users\nleo\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\nleo\.keras-ocr\crnn_kurapan.h5
1/1 [==============================] - 2s 2s/step
box ('arana', array([[509.64456, 395.7455 ],
       [565.9792 , 370.49203],
       [570.5614 , 380.71384],
       [514.22675, 405.96732]], dtype=float32))


KeyboardInterrupt: 

In [6]:
import os
import cv2
import numpy as npFlipping is F
import requests
import math
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageRegionCreateEntry
from msrest.authentication import ApiKeyCredentials
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array
from dotenv import load_dotenv
import keras_ocr

pipeline = keras_ocr.pipeline.Pipeline()

load_dotenv()

training_key = os.getenv('training_key')
endpoint = os.getenv('endpoint')
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
project_id = os.getenv('project_id')
trainer = CustomVisionTrainingClient(endpoint, credentials)

def add_gaussian_noise(image, mean=0, var=10):
    row, col, ch = image.shape
    sigma = var ** 0.5
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = image + gauss
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)

    return noisy

def draw_boxes(image, prediction_groups):
    for box in prediction_groups[0]:
        points = np.array([box[1][0], box[1][1], box[1][2], box[1][3]], dtype=np.int32)
        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
    return image

def inpaint_text(image, pipeline):
    temp_image = image.copy()  # Create a copy to avoid modifying the original image
    prediction_groups = pipeline.recognize([temp_image])
    mask = np.zeros(temp_image.shape[:2], dtype="uint8")
    for box in prediction_groups[0]:

        # points = np.array([box[1][0], box[1][1], box[1][3]], dtype=np.int32)
        # cv2.fillConvexPoly(mask, points, 255)

        #Old
        x0, y0 = box[1][0]
        x1, y1 = box[1][1] 
        x2, y2 = box[1][2]
        x3, y3 = box[1][3] 
        
        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mid1 = midpoint(x0, y0, x3, y3)
        
        thickness = int(math.sqrt((x2 - x1)**2 + (y2 - y1)**2))
        
        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mid1), (220, 220, 220), thickness)
    
    # Apply inpainting
    masked_image = cv2.inpaint(temp_image, mask, 7, cv2.INPAINT_TELEA)
    # cv2.imshow('window', masked_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return masked_image

def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2) / 2)
    y_mid = int((y1 + y2) / 2)
    return x_mid, y_mid

def return_as_dict(tag):
    return tag.as_dict()

remote_tags = trainer.get_tags(project_id)
tags = list(map(return_as_dict, remote_tags))
tag_RA660 = {}

for tag in tags:
    if tag['name'] == 'Tremetrics_RA660':
        tag_RA660 = tag

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)
images = trainer.get_images(project_id, tag_ids=[tag_RA660['id']], order_by=None, tagging_status="Tagged")
image_dir = "C:\\Users\\nleo\\Pictures\\Training data\\Tremetrics\\Tremetrics_RA660"
# original_dir = f'{image_dir}\\original'
masked_dir = f'{image_dir}\\masked_blurred'
# os.makedirs(original_dir, exist_ok=True)
os.makedirs(masked_dir, exist_ok=True)

image_path = f'C:\\Users\\nleo\\Pictures\\Training data\\Tremetrics\\276-to-right.jpg'
image = cv2.imread(image_path)
image_array = img_to_array(image)
image_array = image_array.reshape((1, ) + image_array.shape)

i = 0
for batch in datagen.flow(image_array, batch_size=1):
    
    augmented_image = batch[0].astype('uint8')
    keras_read_image = keras_ocr.tools.read(augmented_image)
    masked_image = inpaint_text(keras_read_image, pipeline)

    var = np.random.choice([10, 20, 30])
    noisy_image = add_gaussian_noise(masked_image, var=var)
    output_path = os.path.join(masked_dir, f'masked{var}_{i}RA660.jpg')

    
    cv2.imwrite(output_path, noisy_image)
    i+=1
    if i > 99:
        break

Looking for C:\Users\nleo\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\nleo\.keras-ocr\crnn_kurapan.h5


UnboundLocalError: cannot access local variable 'image' where it is not associated with a value